In [6]:
# Setup packages and file paths 
import requests
import pandas as pd
import time 

path = "/home/teijehidde/Documents/Git Blog and Coding/data_dump/"
data_file = "data_new3.json"

In [1]:
def downloadWikiNetwork(node_title, lang = "en"):
    api_endpoint = "https://" + lang + ".wikipedia.org/w/api.php" # fr.wikipedia.org; https://en.wikipedia.org
    wiki_data = []
    print("Starting download of network " + node_title + " in language " + lang + ".")
    
    # 1: download data on the node_title wikipage.
    S = requests.Session()
    params_node_title = {
        "action": "query",
        "titles": node_title,
        "prop": "info|links|langlinks", 
        "plnamespace": 0, 
        "pllimit": 500,
        "lllimit": 500, 
        "format": "json"
    }
    response = S.get(url=api_endpoint, params=params_node_title)
    wiki_data.append(response.json())
    
    while 'continue' in wiki_data[-1].keys():
        params_cont = params_node_title
        if 'plcontinue' in wiki_data[-1]['continue']:
            params_cont["plcontinue"] = wiki_data[-1]['continue']['plcontinue'] 
            print('plcontinue: ' + params_cont["plcontinue"])

        elif 'llcontinue' in wiki_data[-1]['continue']:
            params_cont["llcontinue"] = wiki_data[-1]['continue']['llcontinue'] 
            print('llcontinue: ' + params_cont["llcontinue"])

        response = S.get(url=api_endpoint, params=params_cont)
        wiki_data.append(response.json())

    # 2: download data on the links of node_title wikipage.
    S = requests.Session()
    params_network_title = {
        "action": "query",
        "generator": "links",
        "titles": node_title,
        "gplnamespace": 0, 
        "gpllimit": 500, 
        "plnamespace": 0,
        "pllimit": 500, 
        "prop": "info|links|langlinks", 
        "format": "json"
    }
    response = S.get(url=api_endpoint, params=params_network_title)
    wiki_data.append(response.json())

    while 'continue' in wiki_data[-1].keys():
        params_cont = params_network_title
        if 'plcontinue' in wiki_data[-1]['continue']:
            params_cont["plcontinue"] = wiki_data[-1]['continue']['plcontinue'] 
            print('plcontinue: ' + params_cont["plcontinue"])

        elif 'llcontinue' in wiki_data[-1]['continue']:
            params_cont["llcontinue"] = wiki_data[-1]['continue']['llcontinue'] 
            print('llcontinue: ' + params_cont["llcontinue"])

        elif 'gplcontinue' in wiki_data[-1]['continue']: 
            params_cont["gplcontinue"] = wiki_data[-1]['continue']['gplcontinue']
            print('gplcontinue: ' + params_cont["gplcontinue"])

        response = S.get(url=api_endpoint, params = params_cont)
        wiki_data.append(response.json())

    # 3: creating list of available nodes in wiki_data. 
    all_nodes = []

    for item in wiki_data:
        all_nodes = all_nodes + list(item['query']['pages'].keys())
    all_nodes = list(set(all_nodes))
    all_nodes = [int(i) for i in all_nodes if int(i) > 0]
    all_nodes = [str(i) for i in all_nodes]
    
    network_data_df = pd.DataFrame(
        columns = ['title', 'lang', 'pageid', 'uniqueid', 'lastrevid', 'links', 'langlinks', 'ego'], # complete list: ['ns', 'title', 'missing', 'contentmodel', 'pagelanguage', 'pagelanguagehtmlcode', 'pageid', 'lastrevid', 'length', 'links', 'langlinks']
        index = all_nodes)

    # 4: Using all_nodes to go through list of raw data from API. 
    for node in all_nodes:
        if node == node_title: 
            network_data_df.at[node, 'ego'] == True
        network_data_df.at[node,'links'] = []
        for item in wiki_data:
            if node in item['query']['pages'].keys(): # possibility:  df_new_wiki_data.update(item) #, errors = 'raise') 
                network_data_df.at[node, 'title'] = item['query']['pages'][node]['title']
                network_data_df.at[node,'lang'] = item['query']['pages'][node]['pagelanguage']
                network_data_df.at[node,'pageid'] = item['query']['pages'][node]['pageid']
                network_data_df.at[node,'uniqueid'] = network_data_df.at[node,'lang'] + str(network_data_df.at[node,'pageid'])
                network_data_df.at[node,'lastrevid'] = item['query']['pages'][node]['lastrevid']

                if 'links' in item['query']['pages'][node].keys():
                    for link in item['query']['pages'][node]['links']:
                        network_data_df.at[node,'links'].append(link['title'])

                if 'langlinks' in item['query']['pages'][node].keys():
                    network_data_df.at[node,'langlinks'] = item['query']['pages'][node]['langlinks']

    # returns panda with all data from network. 
    return network_data_df

# Function B: Downloading multiple languages of one topic and saving them to json/panda file. 
def downloadMultiLangWikiNetwork(node_title, original_lang = 'en', additional_langs = ["ar", "de", "fr", "nl"]): # or: 'available_langs'
    network_data_df = downloadWikiNetwork(node_title=node_title, lang=original_lang)
    available_langs = network_data_df.loc[network_data_df['langlinks'].notnull()]['langlinks'].values.tolist()[0]

    if additional_langs == []:
        print('The wikipedia page is available in the following languages:')         
        print(available_langs)
    
    else:
        for item in available_langs: 
            if item['lang'] in additional_langs:
                network_data_df_additional = downloadWikiNetwork(node_title = item['*'], lang = item['lang'])
                network_data_df = pd.concat([network_data_df, network_data_df_additional], ignore_index=True, sort=False)
                
    try: 
        network_data_saved = pd.read_json((path + data_file), orient='split')
    except:
        network_data_saved = None
    network_data_df = pd.concat([network_data_df, network_data_saved], ignore_index=True, sort=False)
    network_data_df = network_data_df.loc[network_data_df.astype(str).drop_duplicates(subset=['title', 'lang', 'pageid'], keep = 'first').index].reset_index(drop=True)
    network_data_df.to_json((path + data_file), orient='split')
    print("Download of network and additional languages finished. Returning to main menu...") 
    time.sleep(5) 

In [7]:
data_vaccine = downloadWikiNetwork('Vaccine')

Starting download of network Vaccine in language en.
plcontinue: 32653|0|Serum_Institute_of_India
plcontinue: 737|0|Cage_(organisation)
plcontinue: 737|0|Jamrud_Fort
plcontinue: 737|0|Pew_Research_Center
plcontinue: 737|0|WordNet
plcontinue: 1805|0|Macrophages
plcontinue: 1915|0|Protein_G
plcontinue: 2246|0|Pethidine
plcontinue: 2362|0|Parvalbumin
plcontinue: 2388|0|Parkinson's_disease
plcontinue: 2690|0|Dissociative
plcontinue: 2767|0|Radiopharmacology
plcontinue: 2769|0|Orciprenaline
plcontinue: 2869|0|Benzodiazepine
plcontinue: 2870|0|Disruptive_behavior_disorder
plcontinue: 3997|0|Alveolar_macrophage
plcontinue: 3997|0|Vel_blood_group
plcontinue: 4099|0|Lateral_sacral_crest
plcontinue: 4191|0|2020_Russia–Saudi_Arabia_oil_price_war
plcontinue: 4191|0|COVID-19_pandemic_in_Uttarakhand
plcontinue: 4191|0|List_of_unproven_methods_against_COVID-19
plcontinue: 4502|0|Argentina
plcontinue: 6811|0|Appropriations_bill_(United_States)
plcontinue: 7172|0|American_Cancer_Society
plcontinue: 717

In [1]:
data_vaccine

NameError: name 'data_vaccine' is not defined

In [5]:
# Kubernetes, Vaccine, Oxford, [Verkhovna_Rada this one seems to have a bug - fix later; see also 'Consumer_protection'] 
# wiki_data_kubs = downloadWikiNetwork('Kubernetes')
# wiki_data_Verkhovna_Rada = downloadWikiNetwork('Verkhovna_Rada')

In [27]:
wiki_data = Vaccine_data[0]
all_nodes = []

for item in wiki_data:
    all_nodes = all_nodes + list(item['query']['pages'].keys())
all_nodes = list(set(all_nodes))
all_nodes = [int(i) for i in all_nodes if int(i) > 0]
all_nodes = [str(i) for i in all_nodes]

network_data_df = pd.DataFrame(
    columns = ['title', 'lang', 'pageid', 'uniqueid', 'lastrevid', 'links', 'langlinks'], # complete list: ['ns', 'title', 'missing', 'contentmodel', 'pagelanguage', 'pagelanguagehtmlcode', 'pageid', 'lastrevid', 'length', 'links', 'langlinks']
    index = all_nodes)

# 4: Using all_nodes to go through list of raw data from API. 
for node in all_nodes:
    network_data_df.at[node,'links'] = []
    for item in wiki_data:
        if node in item['query']['pages'].keys(): # possibility:  df_new_wiki_data.update(item) #, errors = 'raise') 
            network_data_df.at[node, 'title'] = item['query']['pages'][node]['title']
            network_data_df.at[node,'lang'] = item['query']['pages'][node]['pagelanguage']
            network_data_df.at[node,'pageid'] = item['query']['pages'][node]['pageid']
            network_data_df.at[node,'uniqueid'] = network_data_df.at[node,'lang'] + str(network_data_df.at[node,'pageid'])
            network_data_df.at[node,'lastrevid'] = item['query']['pages'][node]['lastrevid']

            if 'links' in item['query']['pages'][node].keys():
                for link in item['query']['pages'][node]['links']:
                    network_data_df.at[node,'links'].append(link['title'])

            if 'langlinks' in item['query']['pages'][node].keys():
                network_data_df.at[node,'langlinks'] = item['query']['pages'][node]['langlinks']

In [32]:
len(network_data_df.loc[network_data_df['title'] == 'Vaccine']['links'].values.tolist()[0])

609

In [25]:
# all_nodes

In [26]:
for item in wiki_data['query']['pages']: 
    if '2646623' in item['query']['pages'].keys():
       if 'links' in item['query']['pages']['2646623'].keys(): 
           print(item)

TypeError: list indices must be integers or slices, not str